In [6]:
import os
import sys
import argparse
import numpy as np
# import torch
import pandas as pd
import nibabel as nib    
from pathlib import Path
from surface_distance import metrics
from tqdm import tqdm
# import torch.nn as nn
import ants
import json
import ants

In [7]:
def pred_invert_resample(path_to_pred, path_to_orig, path_to_resampled):
    # path_to_pred = *npz, path_to_orig = *1_reg, path_to_resampled = *4a_resample
    # *.npz archives sometimes can be recognised wrong, if extracted and saved back at the same time
#     data = np.load(path_to_pred, allow_pickle=True)['arr_0']
    # schw
    if np.shape(data)[0] == 1:
        data = data.transpose(0,3,2,1).astype('float32')
        old_orig = ants.image_read(path_to_resampled)
        new_orig = ants.image_read(path_to_orig)
        old_like = old_orig.new_image_like(data[0])
        new_img = ants.resample_image(old_like, new_orig.spacing, False, 0)
        output_file = new_img.numpy().astype('float16')
    
    # gbm and lgg
    elif np.shape(data)[0] > 1:
        data = data.transpose(0,3,2,1).astype('float32')

        old_orig_ct1 = ants.image_read(path_to_resampled)
        new_orig_ct1 = ants.image_read(path_to_orig)

        old_like_ch_0 = old_orig_ct1.new_image_like(data[0])
        old_like_ch_1 = old_orig_ct1.new_image_like(data[1])
        old_like_ch_2 = old_orig_ct1.new_image_like(data[2])

        new_img_0 = ants.resample_image(old_like_ch_0, new_orig_ct1, False, 0)
        new_img_1 = ants.resample_image(old_like_ch_1, new_orig_ct1, False, 0)
        new_img_2 = ants.resample_image(old_like_ch_2, new_orig_ct1, False, 0)

        new_img_shape =  new_img_2.numpy().shape

        new_array = np.zeros(tuple([3] + list(new_img_shape)), dtype='float16')
        new_array[0] = new_img_0.numpy()
        new_array[1] = new_img_1.numpy()
        new_array[2] = new_img_2.numpy()
        output_file = new_array.transpose(0,3,2,1).astype('float16')
    return (output_file)

def pred_invert_resample_1cl(data, path_to_orig, path_to_resampled, resamp_sub, mod):
    # path_to_pred = *npz, path_to_orig = *1_reg, path_to_resampled = *4a_resample
    # *.npz archives sometimes can be recognised wrong, if extracted and saved back at the same time
#     data = np.load(path_to_pred, allow_pickle=True)['arr_0']
    # schw
        data = data.transpose(0,3,2,1).astype('float32')
        old_orig = ants.image_read(path_to_resampled)
        new_orig = ants.image_read(path_to_orig)
        old_like = old_orig.new_image_like(data[0])
        new_img = ants.resample_image(old_like, new_orig.spacing, False, 0)
        output_file = new_img.numpy().astype('float16')
        if new_orig.shape[0]-output_file.shape[0] < 0:
            output_file = output_file[:new_orig.shape[0], :new_orig.shape[1],:new_orig.shape[2] ]
        elif new_orig.shape[0]-output_file.shape[0] > 0:
            output_file = np.pad(output_file, ((0, new_orig.shape[0]-output_file.shape[0]), (0, new_orig.shape[1]-output_file.shape[1]), (0, new_orig.shape[2]-output_file.shape[2])), 'constant', constant_values=0)

        return (output_file)
    
def pred_invert_resample_classes(data, mat_file_path, path_to_orig, path_to_resampled, mod):

    # gbm and lgg
        data = data.transpose(0,3,2,1).astype('float32')
#         print(data.shape)
        old_orig_ct1 = ants.image_read(path_to_resampled)
        new_orig_ct1 = ants.image_read(path_to_orig)
    
#         print(old_orig_ct1.shape)
#         print(new_orig_ct1.shape)
        old_like_ch_0 = old_orig_ct1.new_image_like(data[0])
        old_like_ch_1 = old_orig_ct1.new_image_like(data[1])
        old_like_ch_2 = old_orig_ct1.new_image_like(data[2])
#         print('old_like')
# #         print(old_like_ch_2.shape)
        if mod == '2a_interp':
            new_img_0 = ants.resample_image(old_like_ch_0, new_orig_ct1.shape, True, 0)
            new_img_1 = ants.resample_image(old_like_ch_1, new_orig_ct1.shape, True, 0)
            new_img_2 = ants.resample_image(old_like_ch_2, new_orig_ct1.shape, True, 0)
        if mod == '3a_atlas':
            new_img_0 = ants.apply_transforms(new_orig_ct1, old_like_ch_0, whichtoinvert=[True],
                                          transformlist = mat_file_path)
            new_img_1 = ants.apply_transforms(new_orig_ct1, old_like_ch_1, whichtoinvert=[True],
                                          transformlist = mat_file_path)
            new_img_2 = ants.apply_transforms(new_orig_ct1, old_like_ch_2, whichtoinvert=[True],
                                          transformlist = mat_file_path)
        else:
            new_img_0 = ants.resample_image(old_like_ch_0, new_orig_ct1.spacing, False, 0)
            new_img_1 = ants.resample_image(old_like_ch_1, new_orig_ct1.spacing, False, 0)
            new_img_2 = ants.resample_image(old_like_ch_2, new_orig_ct1.spacing, False, 0)
       
        new_img_shape =  new_img_2.numpy().shape

        new_array = np.zeros(tuple([3] + list(new_img_shape)), dtype='float16')
        new_array[0] = new_img_0.numpy()
        new_array[1] = new_img_1.numpy()
        new_array[2] = new_img_2.numpy()
        output_file = new_array.astype('float16')
#         print(output_file.shape)
#         print('check_shape')
#         print(new_orig_ct1.shape)
#         print(new_orig_ct1.shape[2]-output_file.shape[3])
#         print((new_orig_ct1.shape[1]-output_file.shape[2]))
#         print((new_orig_ct1.shape[0]-output_file.shape[1]))
        if (new_orig_ct1.shape[2]-output_file.shape[3] < 0 ) or (new_orig_ct1.shape[1]-output_file.shape[2] < 0) or (new_orig_ct1.shape[0]-output_file.shape[1] < 0):
            print(1)
            output_file = output_file[:, :new_orig_ct1.shape[0], :new_orig_ct1.shape[1],:new_orig_ct1.shape[2] ]
            
        if (new_orig_ct1.shape[2]-output_file.shape[3] > 0) or (new_orig_ct1.shape[1]-output_file.shape[2] > 0 ) or (new_orig_ct1.shape[0]-output_file.shape[1] > 0):
            print(2)
            output_file = np.pad(output_file, ((0,0), (0, new_orig_ct1.shape[0]-output_file.shape[1]), (0, new_orig_ct1.shape[1]-output_file.shape[2]), (0, new_orig_ct1.shape[2]-output_file.shape[3])), 'constant', constant_values=0)

        output_file = output_file[:,:new_orig_ct1.shape[0], :new_orig_ct1.shape[1],:new_orig_ct1.shape[2] ]
#         print(output_file.shape)
        
        return (output_file)

In [8]:
def calculate_metrics(subjects, path_to_file, path_to_orig, path_to_pred, path_to_resamp, path_to_target, dataset, out = '/home/polina/glioma/all_dice_metrics.csv', mod=None ):
    
    """ 
    - path_to_pred - path to folder with predict subjects
    - path_to_target - path to folder with target subjects
    - name_pred - name for prediction, ex -brainTumorMask_SRI.nii.gz
    - name_target - name for targets, ex -GTV_to_SRI.nii.gz
    - spaces - if false - [1,1,1]
    - name_csv - name files for each subjects
    - path_csv_all - path to the main file with metrics for each subjects
    """
    _columns = ['Ids','Dice_1', 'Dice_2', 'Dice_3',
                'Hausdorff95_1', 'Hausdorff95_2', 'Hausdorff95_3',
                'Sensitivity_1', 'Sensitivity_2', 'Sensitivity_3',
               'Specificity_1', 'Specificity_2', 'Specificity_3',
               'Surface_dice_1', 'Surface_dice_2', 'Surface_dice_3',
               'Precision_1', 'Precision_2', 'Precision_3']
#     _columns = ['Ids','Dice_1'
#                 'Hausdorff95_1',
#                 'Sensitivity_1',
#                'Specificity_1',
#                'Surface_dice_1',
#                'Precision_1']
    
    af_all = pd.DataFrame(columns = _columns)
    pred_folder = Path(path_to_pred)
    orig_folder = Path(path_to_orig)
    resamp_folder = Path(path_to_resamp)
    target_folder = Path(path_to_target)
    file_folder = Path(path_to_file)
    for ids in tqdm(subjects):
        print(ids)
        pred_sub = os.path.join(pred_folder, ids + '.npy.npz')
        orig_sub = os.path.join(orig_folder, ids,'CT1_SEG.nii.gz')
        resamp_sub = os.path.join(resamp_folder, ids,'CT1_SEG.nii.gz')
        mat_fie_sub = os.path.join(file_folder, ids,'T1C_to_SRI_inv.mat')
#         targets = nib.load(os.path.join(target_folder, ids + '_seg.nii.gz'))
#         targets = ants.image_read(orig_sub)
#         targets = ants.reorient_image2(label_orig, orientation = 'LAI')
#         targets = nib.load(orig_sub)
#         spaces = targets.header.get_zooms()
#         spaces = targets.spacing
#         targets = targets.numpy().astype('int')
        targets = ants.image_read(f'{target_folder}/{ids}/CT1_SEG.nii.gz')
        spaces = targets.spacing
#         targets = ants.reorient_image2(targets, orientation = 'LAI').numpy()
        targets = targets.numpy()
#         print(targets.shape)
#         print(np.unique(targets))
        data = np.load(pred_sub, allow_pickle=True)['arr_0']
#         print(data.shape)
        if np.shape(data)[0] == 1: 
            prediction = pred_invert_resample_1cl(data, mat_fie_sub, orig_sub, resamp_sub, mod)
            prediction = np.round(prediction, 0)
#         pred = np.transpose(pred, (0, 3, 2, 1))
#             print(prediction.shape)
#             print(np.unique(prediction))
            df = calculate_metrics_brats_1cl(targets.astype('int'), prediction.astype('int'), ids, spaces)
#             print(df)
        elif np.shape(data)[0] > 1: 
            prediction = pred_invert_resample_classes(data, mat_fie_sub, orig_sub, resamp_sub, mod)
#             prediction = data.transpose(0,3,2,1).astype('float32')
            prediction = np.round(prediction, 0)
            y_wt, y_tc, y_et = targets > 0, ((targets == 1) + (targets == 3)) > 0, targets == 3
            targets = np.stack([y_wt, y_tc, y_et], axis=0).astype(int)
            print(targets.shape)
#             print(prediction.shape)
            df=calculate_metrics_brats(targets.astype('int'), prediction.astype('int'), ids, spaces)
#             except:
#                 continue
        os.makedirs(os.path.join(out, dataset,ids), exist_ok = True)
        out_path = os.path.join(out, dataset,ids, path_to_pred.split('/')[-2] + '_'+ path_to_pred.split('best_')[-1].replace('=', '_') + '.json')
       #         df.to_json(out_path, orient='records')
        sub_dict = {dataset : {path_to_pred.split('/')[-2]: {path_to_pred.split('_')[-2]: {}}}}
        sub_dict[dataset][path_to_pred.split('/')[-2]][path_to_pred.split('_')[-2]] = df.to_dict('records')[0]
#         print(sub_dict)
#         with open(out_path, 'w') as fp:
#             json.dump(sub_dict, fp)
#         print(df)
        af_all = af_all.append(df)
#     af_all.to_csv(out)  
    print(af_all.mean())
    print(len(af_all))
    return (sub_dict)

In [9]:
def pred_invert_resample_classes(data, mat_file_path, path_to_orig, path_to_resampled, mod):

# gbm and lgg
    data = data.transpose(0,3,2,1).astype('float32')
#         print(data.shape)
    old_orig_ct1 = ants.image_read(path_to_resampled)
    new_orig_ct1 = ants.image_read(path_to_orig)

#         print(old_orig_ct1.shape)
#         print(new_orig_ct1.shape)
    old_like_ch_0 = old_orig_ct1.new_image_like(data[0])
    old_like_ch_1 = old_orig_ct1.new_image_like(data[1])
    old_like_ch_2 = old_orig_ct1.new_image_like(data[2])
#         print('old_like')
# #         print(old_like_ch_2.shape)
    if mod == '2a_interp':
        new_img_0 = ants.resample_image(old_like_ch_0, new_orig_ct1.shape, True, 0)
        new_img_1 = ants.resample_image(old_like_ch_1, new_orig_ct1.shape, True, 0)
        new_img_2 = ants.resample_image(old_like_ch_2, new_orig_ct1.shape, True, 0)
    if mod == '3a_atlas':
        new_img_0 = ants.apply_transforms(new_orig_ct1, old_like_ch_0, whichtoinvert=[True],
                                      transformlist = mat_file_path)
        new_img_1 = ants.apply_transforms(new_orig_ct1, old_like_ch_1, whichtoinvert=[True],
                                      transformlist = mat_file_path)
        new_img_2 = ants.apply_transforms(new_orig_ct1, old_like_ch_2, whichtoinvert=[True],
                                      transformlist = mat_file_path)
    else:
        new_img_0 = ants.resample_image(old_like_ch_0, new_orig_ct1.spacing, False, 0)
        new_img_1 = ants.resample_image(old_like_ch_1, new_orig_ct1.spacing, False, 0)
        new_img_2 = ants.resample_image(old_like_ch_2, new_orig_ct1.spacing, False, 0)

    new_img_shape =  new_img_2.numpy().shape

    new_array = np.zeros(tuple([3] + list(new_img_shape)), dtype='float16')
    new_array[0] = new_img_0.numpy()
    new_array[1] = new_img_1.numpy()
    new_array[2] = new_img_2.numpy()
    output_file = new_array.astype('float16')
#         print(output_file.shape)
#         print('check_shape')
#         print(new_orig_ct1.shape)
#         print(new_orig_ct1.shape[2]-output_file.shape[3])
#         print((new_orig_ct1.shape[1]-output_file.shape[2]))
#         print((new_orig_ct1.shape[0]-output_file.shape[1]))

    # checks
    if (new_orig_ct1.shape[2]-output_file.shape[3] < 0 ) or (new_orig_ct1.shape[1]-output_file.shape[2] < 0) or (new_orig_ct1.shape[0]-output_file.shape[1] < 0):
        print(1)
        output_file = output_file[:, :new_orig_ct1.shape[0], :new_orig_ct1.shape[1],:new_orig_ct1.shape[2] ]

    if (new_orig_ct1.shape[2]-output_file.shape[3] > 0) or (new_orig_ct1.shape[1]-output_file.shape[2] > 0 ) or (new_orig_ct1.shape[0]-output_file.shape[1] > 0):
        print(2)
        output_file = np.pad(output_file, ((0,0), (0, new_orig_ct1.shape[0]-output_file.shape[1]), (0, new_orig_ct1.shape[1]-output_file.shape[2]), (0, new_orig_ct1.shape[2]-output_file.shape[3])), 'constant', constant_values=0)

    output_file = output_file[:,:new_orig_ct1.shape[0], :new_orig_ct1.shape[1],:new_orig_ct1.shape[2] ]
#         print(output_file.shape)

    return (output_file)

In [10]:
os.listdir('/mnt/newdata/gbm_infer/')

['gbm_4b_n4_300',
 'gbm_1_reg_train_from_bgpd_TL',
 'gbm_4d_susan_from_bgpd_TL',
 'gbm_5_ss_shared_from_bgpd_TL',
 'gbm_6_histogram_unetr_300',
 '6_histogram_fold_2_from_bgpd_TL',
 'gbm_4d_susan_unetr_300',
 'gbm_5_ss_shared_from_egd_TL',
 'gbm_3a_atlas_from_bgpd_TL',
 'gbm_4a_resamp_unetr_param_wo_aug',
 '6_histogram_fold_0_from_bgpd_TL',
 'gbm_4b_n4_from_bgpd_TL',
 '6_histogram_fold_1_from_bgpd_TL',
 'gbm_3a_atlas_unetr',
 'gbm_6_histogram_300',
 'gbm_3a_atlas_from_brats_TL',
 'gbm_4b_n4_from_bgpd_TL_last_bgpd',
 'gbm_2a_interp_from_bgpd_TL',
 'gbm_5_ss_shared_unetr_300',
 'gbm_2a_interp_unetr',
 'gbm_1_reg_train',
 'gbm_2a_interp_300',
 'gbm_4b_n4_unetr_300',
 'gbm_1_reg_train_unetr',
 'gbm_4a_resamp_from_bgpd_TL',
 'gbm_4a_resamp_ct1_t2',
 'gbm_3a_atlas_300',
 'gbm_3a_atlas_from_egd_TL',
 'gbm_4d_susan_300']

In [11]:
orig_folder = '/anvar/public_datasets/preproc_study/gbm/orig'
resample_folder = '/anvar/public_datasets/preproc_study/gbm/4a_resamp'
file_folder = '/anvar/public_datasets/preproc_study/gbm/3a_atlas'
pred_folder = '/mnt/newdata/gbm_infer/gbm_4d_susan_300/'
target_folder = '/anvar/public_datasets/preproc_study/gbm/1_reg'
reg_1 = '/anvar/public_datasets/preproc_study/gbm/1_reg'
out_json = '/results/metrics'

In [12]:
os.listdir('/anvar/public_datasets/preproc_study/gbm/5_ss_shared/TCGA-02-0054')

['T2.nii.gz',
 'FLAIR.nii.gz',
 'T1.nii.gz',
 'CT1_SEG.nii.gz',
 'CT1_mask.nii.gz',
 'CT1.nii.gz']

In [13]:
os.listdir(pred_folder)[0]

'predictions_best_epoch=230-dice_mean=76_72_task=45_fold=0_tta'

In [19]:
os.listdir('/mnt/newdata/gbm_infer/gbm_4d_susan_300/')

['predictions_best_epoch=230-dice_mean=76_72_task=45_fold=0_tta',
 'predictions_best_epoch=199-dice_mean=72_49_task=45_fold=1_tta',
 'predictions_best_epoch=263-dice_mean=75_78_task=45_fold=2_tta']

In [14]:
ids = 'TCGA-02-0054'
path_to_resampled = os.path.join('/anvar/public_datasets/preproc_study/gbm/4d_susan/', ids, 'CT1_SEG.nii.gz')
mat_file_path = os.path.join(file_folder, ids,'T1C_to_SRI_inv.mat')
path_to_orig = os.path.join('/anvar/public_datasets/preproc_study/gbm/orig', ids, 'CT1_SEG.nii.gz')
pred_folder = '/mnt/newdata/gbm_infer/gbm_4d_susan_300/' + os.listdir(pred_folder)[0]
mod = '4d_susan'

# pred_invert_resample_classes(data, mat_file_path, orig_folder, resample_folder, mod)

In [16]:

data = ants.image_read(path_to_resampled)

# gbm and lgg
data = data.transpose(0,3,2,1).astype('float32')
#         print(data.shape)
old_orig_ct1 = ants.image_read(path_to_resampled)
new_orig_ct1 = ants.image_read(path_to_orig)

#         print(old_orig_ct1.shape)
#         print(new_orig_ct1.shape)
old_like_ch_0 = old_orig_ct1.new_image_like(data[0])
old_like_ch_1 = old_orig_ct1.new_image_like(data[1])
old_like_ch_2 = old_orig_ct1.new_image_like(data[2])
#         print('old_like')
# #         print(old_like_ch_2.shape)
if mod == '2a_interp':
    new_img_0 = ants.resample_image(old_like_ch_0, new_orig_ct1.shape, True, 0)
    new_img_1 = ants.resample_image(old_like_ch_1, new_orig_ct1.shape, True, 0)
    new_img_2 = ants.resample_image(old_like_ch_2, new_orig_ct1.shape, True, 0)
if mod == '3a_atlas':
    new_img_0 = ants.apply_transforms(new_orig_ct1, old_like_ch_0, whichtoinvert=[True],
                                  transformlist = mat_file_path)
    new_img_1 = ants.apply_transforms(new_orig_ct1, old_like_ch_1, whichtoinvert=[True],
                                  transformlist = mat_file_path)
    new_img_2 = ants.apply_transforms(new_orig_ct1, old_like_ch_2, whichtoinvert=[True],
                                  transformlist = mat_file_path)
else:
    new_img_0 = ants.resample_image(old_like_ch_0, new_orig_ct1.spacing, False, 0)
    new_img_1 = ants.resample_image(old_like_ch_1, new_orig_ct1.spacing, False, 0)
    new_img_2 = ants.resample_image(old_like_ch_2, new_orig_ct1.spacing, False, 0)

new_img_shape =  new_img_2.numpy().shape

new_array = np.zeros(tuple([3] + list(new_img_shape)), dtype='float16')
new_array[0] = new_img_0.numpy()
new_array[1] = new_img_1.numpy()
new_array[2] = new_img_2.numpy()
output_file = new_array.astype('float16')
#         print(output_file.shape)
#         print('check_shape')
#         print(new_orig_ct1.shape)
#         print(new_orig_ct1.shape[2]-output_file.shape[3])
#         print((new_orig_ct1.shape[1]-output_file.shape[2]))
#         print((new_orig_ct1.shape[0]-output_file.shape[1]))

# checks
if (new_orig_ct1.shape[2]-output_file.shape[3] < 0 ) or (new_orig_ct1.shape[1]-output_file.shape[2] < 0) or (new_orig_ct1.shape[0]-output_file.shape[1] < 0):
    print(1)
    output_file = output_file[:, :new_orig_ct1.shape[0], :new_orig_ct1.shape[1],:new_orig_ct1.shape[2] ]

if (new_orig_ct1.shape[2]-output_file.shape[3] > 0) or (new_orig_ct1.shape[1]-output_file.shape[2] > 0 ) or (new_orig_ct1.shape[0]-output_file.shape[1] > 0):
    print(2)
    output_file = np.pad(output_file, ((0,0), (0, new_orig_ct1.shape[0]-output_file.shape[1]), (0, new_orig_ct1.shape[1]-output_file.shape[2]), (0, new_orig_ct1.shape[2]-output_file.shape[3])), 'constant', constant_values=0)

output_file = output_file[:,:new_orig_ct1.shape[0], :new_orig_ct1.shape[1],:new_orig_ct1.shape[2] ]
#         print(output_file.shape)

NameError: name 'data' is not defined

In [45]:
output_file.shape

(3, 256, 256, 23)

In [46]:
folder_predicts_ss = Path('/anvar/public_datasets/preproc_study/gbm/inference/native_space/5_ss')
folder_predicts_atlas = Path('/anvar/public_datasets/preproc_study/gbm/inference/native_space/3a_atlas')
folds = Path('/home/anvar/projects/brain-mri-processing-pipeline/main_pipeline/utils')
folder_mask = Path('/anvar/public_datasets/preproc_study/gbm/1_reg/')

In [47]:
orig_folder = '/anvar/public_datasets/preproc_study/gbm/orig'
resample_folder = '/data_anvar/public_datasets/preproc_study/gbm/4a_resamp'
file = '/anvar/public_datasets/preproc_study/gbm/3a_atlas'
pred_folder = '/mnt/newdata/gbm_infer/gbm_5_ss_shared_from_bgpd_TL/predictions_best_epoch=40-dice_mean=82_24_task=28_fold=2_tta'
target_folder = '/anvar/public_datasets/preproc_study/gbm/1_reg'
reg_1 = '/anvar/public_datasets/preproc_study/gbm/1_reg'
out_json = '/results/metrics'
dataset = 'gbm'
# TCGA-76-4932
subjects = [each[:-8] for each in os.listdir(pred_folder)]
assert(len(subjects) == 102)
# subjects = ['TCGA-02-0027']

In [48]:
subjects[1]

'TCGA-06-0164'

In [ ]:
calculate_metrics([subjects[0]], file, reg_1, pred_folder, resample_folder, target_folder, dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

In [1]:
ось х в милилитрах, а y в процентах
ошибка в дайсе и нет ошибки в объемах

SyntaxError: invalid syntax (2997777125.py, line 1)

In [2]:
data = np.load(pred_sub, allow_pickle=True)['arr_0']
#         print(data.shape)
if np.shape(data)[0] == 1: 
    prediction = pred_invert_resample_1cl(data, mat_fie_sub, orig_sub, resamp_sub, mod)
    prediction = np.round(prediction, 0)
#         pred = np.transpose(pred, (0, 3, 2, 1))
#             print(prediction.shape)
#             print(np.unique(prediction))
    df = calculate_metrics_brats_1cl(targets.astype('int'), prediction.astype('int'), ids, spaces)
#             print(df)
elif np.shape(data)[0] > 1: 
    prediction = pred_invert_resample_classes(data, mat_fie_sub, orig_sub, resamp_sub, mod)
#             prediction = data.transpose(0,3,2,1).astype('float32')
    prediction = np.round(prediction, 0)
    y_wt, y_tc, y_et = targets > 0, ((targets == 1) + (targets == 3)) > 0, targets == 3
    targets = np.stack([y_wt, y_tc, y_et], axis=0).astype(int)
    print(targets.shape)
#             print(prediction.shape)
    df = calculate_metrics_brats(targets.astype('int'), prediction.astype('int'), ids, spaces)

NameError: name 'np' is not defined